In [0]:
from databricks.sdk import WorkspaceClient
from datetime import datetime

# Initialize workspace client
workspace_client = WorkspaceClient()

# Job ID for ADS_RDS_PROC
job_id = "1092027122452197"  # ADS_RDS_PROC
# job_id = "449940996801892"  # Uncomment for ADS_RDS_Counts

# Optional: Set limit for number of runs to retrieve
limit = 26

# Filter: Only show runs from this date onwards (format: YYYY-MM-DD)
run_date_from = "2025-09-30"  # Set to None to show all runs
# run_date_from = None  # Uncomment to show all runs

# Convert filter date to datetime object for comparison
filter_date = datetime.strptime(run_date_from, '%Y-%m-%d') if run_date_from else None

# Get job details for tags
job_details = workspace_client.jobs.get(job_id=job_id)
job_name = job_details.settings.name

print(f"{'='*100}")
print(f"Job: {job_name} (ID: {job_id})")
if filter_date:
    print(f"Filtering: Runs from {run_date_from} onwards")
print(f"{'='*100}\n")

# Get job runs
runs = workspace_client.jobs.list_runs(job_id=job_id, limit=limit)

# Convert to list and filter by date if needed
filtered_runs = []
for run in runs:
    if run.start_time:
        run_datetime = datetime.fromtimestamp(run.start_time / 1000)
        if filter_date is None or run_datetime >= filter_date:
            filtered_runs.append(run)

# Sort runs by start time (oldest first for chronological order)
filtered_runs.sort(key=lambda x: x.start_time if x.start_time else 0)

if not filtered_runs:
    print(f"No runs found matching the filter criteria.")
else:
    for i, run in enumerate(filtered_runs, 1):
        # Convert epoch milliseconds to datetime
        start_time = datetime.fromtimestamp(run.start_time / 1000).strftime('%Y-%m-%d %H:%M:%S') if run.start_time else "N/A"
        end_time = datetime.fromtimestamp(run.end_time / 1000).strftime('%Y-%m-%d %H:%M:%S') if run.end_time else "N/A"
        
        # Calculate duration
        duration_ms = run.execution_duration if run.execution_duration else 0
        duration_min = duration_ms / 1000 / 60 if duration_ms else 0
        duration_sec = duration_ms / 1000 if duration_ms else 0
        
        # Get parameters
        parameters = {}
        if run.overriding_parameters and run.overriding_parameters.job_parameters:
            parameters = run.overriding_parameters.job_parameters
        
        # Get current tags
        tags = job_details.settings.tags if job_details.settings.tags else {}
        
        print(f"[Run {i}] Run ID: {run.run_id}")
        print(f"  Run Name: {run.run_name}")
        print(f"  State: {run.state.result_state if run.state else 'Unknown'}")
        print(f"  Life Cycle: {run.state.life_cycle_state if run.state else 'Unknown'}")
        print(f"  Start Time: {start_time}")
        print(f"  End Time: {end_time}")
        print(f"  Duration: {duration_min:.2f} minutes ({duration_sec:.1f} seconds)")
        
        if parameters:
            print(f"  Parameters:")
            for key, value in parameters.items():
                print(f"    {key}: {value}")
        
        if tags:
            print(f"  Tags:")
            for key, value in tags.items():
                print(f"    {key}: {value}")
        
        print(f"  URL: {run.run_page_url}")
        print(f"{'-'*100}\n")

    print(f"{'='*100}")
    print(f"Total runs displayed: {len(filtered_runs)}")
    print(f"{'='*100}")